[View in Colaboratory](https://colab.research.google.com/github/laziszaire/notebooks/blob/dev/minimal_nmt.ipynb)

In [1]:
!echo -e "这是只猫\n那是条狗" > source
!cat source
!printf "This is a cat\nThat's a dog" > target
!cat target

这是只猫
那是条狗
This is a cat
That's a dog

In [0]:
import tensorflow as tf

Iterator:  
数据输入管道Data Input Pipeline

In [0]:
#ETL: Extract
ds_source = tf.data.TextLineDataset("source")
ds_target = tf.data.TextLineDataset("target")

In [4]:
#ETL: Load
target_iter = ds_target.make_one_shot_iterator()
with tf.Session() as sess:
  print(target_iter.get_next().eval().decode('utf-8'))

This is a cat


In [0]:

# dataset = ds_target.map(lambda string: tf.string_split([string]).values)
# dataset = dataset.map(lambda words: (words, tf.size(words)))
# _iter = dataset.make_one_shot_iterator()
# words, length = _iter.get_next()
# vocab = []
# # build vocabulary: tensorflow
# with tf.Session() as sess:
#   while True:
#     try:
#       _words, _length = sess.run([words, length])
#       vocab.append(_words)
#     except tf.errors.OutOfRangeError:
#       break
#   vocab = tf.unique(tf.concat(vocab, 0))
#   print(vocab.eval())
#   table = tf.contrib.lookup.index_table_from_tensor(vocab)
# #因此不需要decode

# dataset = dataset.map(lambda words, size: (table.lookup(words), size))
# _iter= dataset.make_initializable_iterator()
# init = _iter.initializer
# wordid, length = _iter.get_next()
# with tf.Session() as sess:
#   sess.run(init)
#   tf.tables_initializer().run()
#   print(sess.run([wordid, length]))

In [0]:
# 抄就行了啊 别自己造轮子

In [7]:
# build vocabulary
# python
import string
def vocab(files, language='en'):
  cab = set()
  for file in files:
    with open(file, 'r') as f:
      text = f.read()
      if language =='en':
        words = set([word.strip(string.punctuation) for word in text.split()])
      elif language =='ch':
        words = set([char for char in text if char is not '\n'])
      else:
        raise ValueError('Not supported')
      cab = cab.union(words)
  return {word: idx for idx, word in enumerate(cab)}

def inver_vocab(cab):
  return {idx: word for word, idx in cab.items()}

source_vocab = vocab(['source'], language='ch')
print(source_vocab)
target_vocab = vocab(['target'], language='en')
print(target_vocab)
print(inver_vocab(target_vocab))

{'那': 0, '是': 1, '条': 2, '只': 3, '狗': 4, '这': 5, '猫': 6}
{'cat': 0, 'is': 1, "That's": 2, 'This': 3, 'dog': 4, 'a': 5}
{0: 'cat', 1: 'is', 2: "That's", 3: 'This', 4: 'dog', 5: 'a'}


In [0]:
with open('source', encoding='utf-8') as f:
  ss = []
  for line in f:
    ss.append([(char, source_vocab[char]) for char in line if char is not '\n'])

with open('target', encoding='utf-8') as f:
  ts = []
  for line in f:
    ts.append([(word, target_vocab[word]) for word in line.split() if word is not '\n'])

In [0]:
src = ss[0]
schars, sidx = zip(*src)

tsrc = ts[0]
twords, tidx = zip(*tsrc)

In [0]:
src_vac_size = len(source_vocab)
tgt_vac_size = len(target_vocab)
ebd_size = 10
num_units = 11
batch_size = 1
max_gradient_norm = 1
learning_rate = .0001

In [0]:
g = tf.Graph()
with g.as_default():
  
  #encode
  with tf.variable_scope('encoder', reuse=tf.AUTO_REUSE) as scope:
    _src = tf.expand_dims(tf.constant(sidx),0)
    _tgt = tf.expand_dims(tf.constant(tidx),0)
    tlength = tf.expand_dims(tf.size(_tgt), 0)
    seq_length = tf.size(_src)
    ebd = tf.get_variable('src_embdding', 
                          [src_vac_size, ebd_size], 
                          dtype=tf.float32)
    x = tf.nn.embedding_lookup(ebd, _src)
    #bidirectional lstm
    forword_cell = tf.nn.rnn_cell.LSTMCell(num_units, name='Fbasic_lstm_cell')
    backword_cell = tf.nn.rnn_cell.LSTMCell(num_units, name='Bbasic_lstm_cell')
    bi_outputs, encoder_state = tf.nn.bidirectional_dynamic_rnn(
    forword_cell, backword_cell, x, dtype=tf.float32)
    encoder_outputs = tf.concat(bi_outputs, -1)
    
  #decode
  with tf.variable_scope('decoder', reuse=tf.AUTO_REUSE) as scope:
    decoder_cell = tf.nn.rnn_cell.LSTMCell(2*num_units)
    t_ebd = tf.get_variable('tgt_embdding', 
                          [tgt_vac_size, ebd_size], 
                          dtype=tf.float32)
    y = tf.nn.embedding_lookup(t_ebd, _tgt)
    helper = tf.contrib.seq2seq.TrainingHelper(y, tlength)
    projection_layer = tf.layers.Dense(tgt_vac_size)
    _encoder_state = tf.nn.rnn_cell.LSTMStateTuple(c=tf.concat([encoder_state[0].c,encoder_state[1].c], -1),
                              h=tf.concat([encoder_state[0].h,encoder_state[1].h], -1))
    decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, 
                                              helper, 
                                              _encoder_state,
                                              output_layer=projection_layer)
    outputs, _, _= tf.contrib.seq2seq.dynamic_decode(decoder)
    logits = outputs.rnn_output
    
  #loss: cross_entropy
  cross_ent = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=_tgt, logits=logits)
  train_loss = tf.reduce_sum(cross_ent)/batch_size
  
  #gradient and optimization
  params = tf.trainable_variables()
  gradients = tf.gradients(train_loss, params)
  clipped_gradients, _ = tf.clip_by_global_norm(gradients, max_gradient_norm)
  optimizer = tf.train.AdamOptimizer(learning_rate)
  update_step = optimizer.apply_gradients(zip(clipped_gradients, params))

In [12]:
import itertools
with tf.Session(graph=g) as sess:
  sess.run(tf.global_variables_initializer())
  for i in itertools.count():
    _, _lossi = sess.run([update_step, train_loss])
    if i % 100 == 0:
      print(_lossi)
    if _lossi < .1:
      _outf = sess.run([logits])
      break

7.258453
6.875393
6.421062
5.8247933
5.0776534
4.270583
3.4546585
2.6514919
1.9349053
1.3539553
0.93535984
0.67111194
0.51112634
0.4045411
0.32889313
0.272713
0.229622
0.19585845
0.16902685
0.14744169
0.12984718
0.11530785
0.103132896


In [0]:
import numpy as np
def translate(idx, vocab):
  return ' '.join(inver_vocab(vocab)[i] for i in idx)

In [14]:
print(translate(sidx, source_vocab))
#翻译
translate(np.argmax(_outf[0][0,:,:], axis=1), target_vocab)

这 是 只 猫


'This is a cat'

# minimal NMT 完成
todo:
- multilayer LSTM
- attention mechanism
- residual block